In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
// Fork-Join network example 8 - fork-join with multiple visits within the same chain
val model = Network("model")

In [ ]:
// Create network
val source = Source(model, "Source")
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
val queue2 = Queue(model, "Queue2", SchedStrategy.PS)
val fork = Fork(model, "Fork")
val join = Join(model, "Join", fork)
val sink = Sink(model, "Sink")

val jobclass1 = OpenClass(model, "class1")
val jobclass2 = OpenClass(model, "class2")

In [ ]:
// Service configurations
source.setArrival(jobclass1, Exp(0.1))
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(1.0))
queue1.setService(jobclass2, Exp(1.0))
queue2.setService(jobclass2, Exp(1.0))

In [ ]:
// Set routing matrix with class switching - multiple visits
val P = model.initRoutingMatrix()

// Class 1 initial routing
P.set(jobclass1, jobclass1, source, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)

// Class switching: class1 -> class2 after join
P.set(jobclass1, jobclass2, join, fork, 1.0)

// Class 2 routing
P.set(jobclass2, jobclass2, fork, queue1, 1.0)
P.set(jobclass2, jobclass2, fork, queue2, 1.0)
P.set(jobclass2, jobclass2, queue1, join, 1.0)
P.set(jobclass2, jobclass2, queue2, join, 1.0)
P.set(jobclass2, jobclass2, join, sink, 1.0)

model.link(P)

In [ ]:
// Solve with multiple methods
val solvers = mutableListOf<NetworkSolver>()

solvers.add(MVA(model))
solvers.add(JMT(model, "seed", 23000))

for ((i, solver) in solvers.withIndex()) {
    println("SOLVER ${i+1}: ${solver.name}")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
}

This fork-join with class switching model demonstrates:

1. **Multiple Fork-Join Visits**:
   - Jobs visit the fork-join structure twice
   - First visit as class1, second visit as class2
   - Creates complex flow patterns through the network

2. **Class Switching Mechanism**:
   - **class1**: Source → Fork → {Queue1, Queue2} → Join → (switch to class2)
   - **class2**: Fork → {Queue1, Queue2} → Join → Sink
   - Join node acts as class switching point

3. **Service Configuration**:
   - All queues serve both classes with rate 1.0
   - PS scheduling provides fair sharing between classes
   - Source generates class1 jobs at rate 0.1

4. **Performance Implications**:
   - **Double utilization**: Each arriving job creates 4 queue visits total
   - **Load amplification**: Fork-join synchronization occurs twice per job
   - **Class interaction**: Both classes compete for resources

5. **Open System Properties**:
   - Continuous arrival of new jobs
   - Jobs eventually leave after two fork-join cycles
   - Steady-state analysis shows cumulative effects

6. **Synchronization Complexity**:
   - Each job requires 4 synchronization points total
   - First sync after class1 processing
   - Second sync after class2 processing

The model represents:
- Multi-stage manufacturing with rework loops
- Software systems with iterative processing phases
- Quality control systems with inspection cycles
- Data processing pipelines with multiple validation stages

Key insight: Class switching in fork-join networks creates multiplicative effects on system load and complexity, as each job generates multiple parallel processing requirements throughout its lifecycle.

**Note**: MVA solver may not support complex class switching patterns, so only JMT (simulation) is used for reliable results.